In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import regex as re
from bs4 import BeautifulSoup
import requests
import urllib
from urllib.request import urlopen
from selenium.common.exceptions import NoSuchElementException
from collections import Counter
import math
import io
from PyPDF2 import PdfReader
import datefinder
import plotly.express as px
from PIL import Image
import pytesseract
import pdf2image
import nltk

c:\users\ramon\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
import requests
from io import BytesIO
from pdf2image import convert_from_bytes

In [3]:
def OCR_PDF(url):
    tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
    tokenizer._params.abbrev_types.update({'d.d'})
    
    pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ramon\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

    pdf = requests.get(url)
    
    images = pdf2image.convert_from_bytes(pdf.content, first_page=1, last_page=1, poppler_path=r"C:\Users\ramon\Downloads\Release-23.01.0-0\poppler-23.01.0\Library\bin")
    
    image = images[0]
    
    image.save('page.jpg', 'JPEG')
    
    string = pytesseract.image_to_string(Image.open('page.jpg'))
    
    string = string.lower()
        
    string = string.replace('\n', ' ')
    sentences = tokenizer.tokenize(string)
    
    return sentences

#print(OCR_PDF('https://open.overheid.nl/documenten/ronl-450993204517c5bb0a905ce3ad989c9ac5c4a4ae/pdf'))

In [98]:
# Hiermee is het mogelijk om pdf's te lezen uit een pdfreader wellicht waardeloos, maar kan in sommige gevallen wellicht 
# wel handig blijken want soms is de verzoekdatum wel makkelijk te achterhalen https://open.overheid.nl/Details/ronl-b1ffb1d37c52ac6e77427cffeaf330381012b589/1#panel-teksterhalen (in uw brief van ... ) om maar een vb te noemen

def verzoek_datum(url, OCR = False):
    geachte_bool = False
    
    tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()

    # ignore d.d. anders wordt dit als het einde van een zin gezien.
    tokenizer._params.abbrev_types.update({'d.d'})
    
    bools = []
    
    r = requests.get(url)
    f = io.BytesIO(r.content)
    
    try:
        reader = PdfReader(f)
        pdf_len = len(reader.pages)

    except:
        return np.nan, 0
    
    # leest eerste pagina enige mogelijk nuttige pagina lijkt mij
    contents = reader.pages[0].extract_text()
    contents = contents.lower()
    
    contents = contents.replace('\n', ' ')
    contents = tokenizer.tokenize(contents)
    
    if contents == []:
        if OCR == False:
            return "image", pdf_len
        
        if OCR == True:
            contents = OCR_PDF(url)
         
    # als wob of woo niet 1x genoemd wordt ga ik ervan uit dat we met een ander soort verzoek te maken hebben.
    
    terms_to_check = ["wob", "woo", "wet open overheid", "wet openbaarheid van bestuur", "wetopenoverheid", "wetopenbaarheidvanbestuur"]
    if not any(word in sentence for sentence in contents for word in terms_to_check):
        return np.nan, pdf_len
    else:
        bools.append(True)
    
    if any("betreft" in s for s in contents):
        bools.append(True)

    if any("datum:" in s for s in contents):
        bools.append(True)
        
    if any("onderwerp" in s for s in contents):
        bools.append(True)
        
    if any("ons kenmerk" in s for s in contents):
        bools.append(True)
        
    if any("retouradres postbus" in s for s in contents):
        bools.append(True)
        
    if any("verzoek" in s for s in contents):
        bools.append(True)
    
    if sum(bools) <= 1:
        return np.nan, pdf_len
    
    # proberen contents te splitsen want de verzoekdatum zal altijd na 'geachte' komen
    contents = ' '.join(contents)
    
    # geen geachte maar geacht want de persoon is vaak gelakt en dat lijd tot slecht leesbare
    contents = contents.split('geachte')

    if len(contents) > 1:
        geachte_bool = True
        contents = contents[1]
    else:
        contents = contents[0]
        
    contents = contents.replace('\n', ' ')
    contents = tokenizer.tokenize(contents)
    
    matching = []
    possible_matches = ['inuw', 'heeftu', 'verzoekingediend', 'uheeft', 'hebbenuw', 'uwverzoek','uwbrief', 'uwmail', 'bijbriefvan', 'middelseenverzoek', 'verzochtu']
    for content in contents:
        content = content.replace(" ", "")
        if any(i in content for i in possible_matches) == True:
            if content != 'wiltuslechtséénzaakinuwbriefbehandelen.':
                matching = content
                break

    if matching == []:
        return np.nan, pdf_len
    
    # weird empty spaces that occur often will be removed
    matching = matching.split(' ')
    if '' in matching:
        matching = [i for i in matching if i != '']
    
    # omdat de datum van het verzoek meestal in de eerste zin staat willen we de eerste match
    matching = ' '.join(matching)
    
    maanden = ['januari','februari','maart', 'april', 'mei', 'juni', 'juli', 'augustus', 'september', 'oktober', 'november', 'december']
    
    reg = re.findall(r"(0?[1-9]|[12][0-9]|3[01])[^0-9.]?(januari|februari|maart|april|mei|juni|juli|september|augustus|oktober|november|december).?(:19[7-9]\d|2\d{3})(?=\D|$)", matching)
        
    if reg == []:
        return np.nan, pdf_len
    
    if 'datum' in matching and len(reg) > 1 and geachte_bool == False:
        reg = reg[1]
    
    else:
        reg = reg[0]

    for n in enumerate(maanden):
        if reg[1] == n[1]:
            reg = list(reg)
            if n[0] < 9:
                reg[1] = '0' + str(n[0] + 1)
            else:
                reg[1] = str(n[0] + 1)
    
    if len(reg[0]) == 1:
        reg[0] = '0' + reg[0]
    
    reg = '-'.join(reg)
    
    return reg, pdf_len

print(verzoek_datum('https://open.overheid.nl/documenten/ronl-8711a9e3de852cecd16d28cc4150c07933a8b9f2/pdf', False))


('08-04-2022', 2)


https://open.overheid.nl/documenten/ronl-c8fdc0ed398409a1dbeae58c46f59d4fee7e0b64/pdf is een ander soort verzoek maar komt ook door de keuring heen.

In [99]:
Dataframe = pd.read_csv('Woo5mei.csv')

display(Dataframe[3040:])

,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp,url,verzoekdatum,Doc_len
3040,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen 11-17 bij besluit op Woo-verzoek over...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-a624d...,NaN,98
3041,Besluit op Woo-verzoek over overeenkomst Arcad...,Besluit op Woo-verzoek over overeenkomst Arcadis,Lijst van documenten,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-a77f8...,NaN,9
3042,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlage 20 bij besluit op Woo-verzoek over ove...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-a97fa...,NaN,82
3043,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen 22-23 bij besluit op Woo-verzoek over...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-c0708...,NaN,100
3044,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen 18-19 bij besluit op Woo-verzoek over...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-c9d13...,NaN,82
3045,Besluit op Woo-verzoek over meststoffen en afv...,Besluit op Woo-verzoek over meststoffen en afv...,NaN,"Ministerie van Landbouw, Natuur en Voedselkwal...",Stoffen,Beslissing op Wob-/Woo-verzoek,06-03-2023,03-03-2023,Mest,https://open.overheid.nl/documenten/ronl-cbaf4...,23-05-2022,42
3046,Besluit Woo-verzoek ECER-website - Openbaar ge...,Besluit Woo-verzoek ECER-website,Openbaar gemaakte documenten deel 1,Ministerie van Buitenlandse Zaken,Europese zaken,Beslissing op Wob-/Woo-verzoek,06-03-2023,08-02-2023,Europese Unie,https://open.overheid.nl/documenten/ronl-da28a...,NaN,260
3047,Besluit op Woo-verzoek over aanvraag tot verpl...,Besluit op Woo-verzoek over aanvraag tot verpl...,Woo-verzoek aanvraag verplichtstelling deelnem...,Ministerie van Sociale Zaken en Werkgelegenheid,"['Arbeidsverhoudingen', 'Arbeidsverhoudingen']",Beslissing op Wob-/Woo-verzoek,06-03-2023,03-03-2023,"['Arbeidsovereenkomst en cao', 'Pensioen']",https://open.overheid.nl/documenten/ronl-e1591...,03-03-2023,7
3048,Bijlagen deel 3 bij besluit op Woo-verzoek ove...,Bijlagen deel 3 bij besluit op Woo-verzoek ove...,Bijlagen 195-197 bij besluit op Woo-verzoek ov...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-e1644...,NaN,128
3049,Bijlagen deel 2 bij besluit op Woo-verzoek ove...,Bijlagen deel 2 bij besluit op Woo-verzoek ove...,Bijlagen 43-50 bij besluit op Woo-verzoek over...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-e36fc...,NaN,98


Deelbesluiten etc omzetten tot gewoon besluit. 

In [103]:
def eerstwrd(woord):
    
    eerste3woorden = woord.split(' ')[:3]
    string = ' '.join(eerste3woorden)
    
    if 'besluit' in string:
        if eerste3woorden[2].isnumeric() == True:
            Split = woord.split(eerste3woorden[2])
        elif eerste3woorden[1].isnumeric() == True:
            Split = woord.split(eerste3woorden[1])    
            
        else:
            Split = woord.split('besluit')
        
        return 'Besluit' + Split[1]
    
    elif 'beslissing' in string:
        
        if eerste3woorden[2].isnumeric() == True:
            Split = woord.split(eerste3woorden[2])
        elif eerste3woorden[1].isnumeric() == True:
            Split = woord.split(eerste3woorden[1])    
        else:
            Split = woord.split('beslissing')
            
        return 'Beslissing' + Split[1]
    
    else:
        return woord

In [104]:
Dataframe['Documentdatum'] = pd.to_datetime(Dataframe['Documentdatum'], format='%d-%m-%Y')
Dataframe = Dataframe.loc[Dataframe['Documentdatum'] >= '01-05-2022']

Dataframe['Verzoek'] = Dataframe['Verzoek'].map(lambda x:eerstwrd(x))

# deze functie zal het wisselende beginwoord proberen te veranderen naar Besluit of Beslissing
Dataframe['eerste_woord'] = Dataframe['Verzoek'].str.split(' ').str.get(0)

# Omdat het moeilijk is om te bepalen welk beginwoord wel of niet te droppen heb ik besloten enkel deze te behouden.
woord_list = ['Besluit', 'Wob-verzoek', 'Beslissing', 'Wob-besluit', 'Woo-verzoek','Woo-besluit']

Dataframe = Dataframe[Dataframe['eerste_woord'].isin(woord_list)]
Dataframe = Dataframe.drop('eerste_woord', axis=1)

display(Dataframe)

,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp,url,verzoekdatum,Doc_len
0,Besluit op Wob-verzoek over aankoop van het sc...,Besluit op Wob-verzoek over aankoop van het sc...,NaN,"Ministerie van Onderwijs, Cultuur en Wetenschap",Cultuur,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,Kunst en cultuur,https://open.overheid.nl/documenten/ronl-0b4b7...,NaN,271
1,Besluit op Wob-verzoek over het op afstand pla...,Besluit op Wob-verzoek over het op afstand pla...,bijlagen bij Wob-verzoek over op afstand plaat...,Ministerie van Financiën,Onbekend thema,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,NaN,https://open.overheid.nl/documenten/ronl-232e7...,NaN,68
2,Besluit op Wob-verzoek over aankoop van het sc...,Besluit op Wob-verzoek over aankoop van het sc...,Inventarislijst Wob-verzoek over aankoop van s...,"Ministerie van Onderwijs, Cultuur en Wetenschap",Cultuur,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,Kunst en cultuur,https://open.overheid.nl/documenten/ronl-2af63...,NaN,18
3,Besluit op Wob-verzoek over aankoop van het sc...,Besluit op Wob-verzoek over aankoop van het sc...,NaN,"Ministerie van Onderwijs, Cultuur en Wetenschap",Cultuur,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,Kunst en cultuur,https://open.overheid.nl/documenten/ronl-3210e...,NaN,209
4,Besluit op Wob-verzoek correspondentie over st...,Besluit op Wob-verzoek correspondentie over st...,wobbesluit-staatssteunaspecten-bik.pdf,Ministerie van Financiën,Inkoop en beheer,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,Staatssteun,https://open.overheid.nl/documenten/ronl-3c18a...,09-07-2021,7
...,...,...,...,...,...,...,...,...,...,...,...,...
3056,Besluit op Woo-verzoek over kennisgeving onder...,Besluit op Woo-verzoek over kennisgeving onder...,Besluit,Ministerie van Infrastructuur en Waterstaat,Onbekend thema,Beslissing op Wob-/Woo-verzoek,07-03-2023,2023-02-02,NaN,https://open.overheid.nl/documenten/ronl-16bf3...,09-12-2022,6
3057,Besluit op Woo-verzoek over kennisgeving onder...,Besluit op Woo-verzoek over kennisgeving onder...,Bijlagen,Ministerie van Infrastructuur en Waterstaat,Onbekend thema,Beslissing op Wob-/Woo-verzoek,07-03-2023,2023-02-02,NaN,https://open.overheid.nl/documenten/ronl-21c18...,NaN,85
3058,Besluit op Woo-verzoek over lozingsvergunning ...,Besluit op Woo-verzoek over lozingsvergunning ...,NaN,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,07-03-2023,2023-03-01,NaN,https://open.overheid.nl/documenten/ronl-37873...,20-02-2023,11
3059,Besluit op Woo-verzoek over uitgezonden percen...,Besluit op Woo-verzoek over uitgezonden percen...,NaN,Ministerie van Economische Zaken en Klimaat,"['Dienstensector', 'Media']",Beslissing op Wob-/Woo-verzoek,07-03-2023,2023-03-03,"['Telecommunicatie', 'Media en publieke omroep']",https://open.overheid.nl/documenten/ronl-b937a...,NaN,31


Enkel besluiten van ministeries hebben de structuur die bruikbaar is. 

In [105]:
def eerste_woord(string):
    spl = string.split(' ')
    if spl[0] != 'Ministerie':
        return np.nan
    else:
        return string

In [106]:
print(len(Dataframe))
Dataframe['Verantwoordelijke'] = Dataframe['Verantwoordelijke'].map(lambda x:eerste_woord(x))
Dataframe = Dataframe[Dataframe['Verantwoordelijke'].notna()]
print(len(Dataframe))

2788
2775


Statistieken over de verzoekdatum:

In [107]:
x = Dataframe['verzoekdatum'].values.tolist()

total = len(x)
nans = len([i for i in x if i is np.nan])
imgs = len([i for i in x if i == 'image'])

print(f'{total} Totaal aantal documenten')
print(f"{nans} Aantal documenten zonder vindbare verzoekdatum")
print(f'{imgs} Documenten die niet leesbaar zijn (images)')

dates = total - nans - imgs
print(str((imgs/total) * 100) + ' % fotos')
print(str((dates/total) * 100) + ' % met verzoekdatum')

2775 Totaal aantal documenten
1466 Aantal documenten zonder vindbare verzoekdatum
334 Documenten die niet leesbaar zijn (images)
12.036036036036037 % fotos
35.13513513513514 % met verzoekdatum


Ik haal op dit moment alle besluiten met meerdere data weg maar er kan in de toekomst gekozen worden om dan de eerste of laatste datum te nemen afhankelijk van wat wenselijk is.

In [108]:
df_urls = Dataframe.groupby(['Verzoek'])['url'].apply(list).reset_index(name='urls')
# deelbesluiten hebben mogelijk meerdere data dus ook deze nemen we niet of later pas mee!
df_documentdatums = Dataframe.groupby(['Verzoek'])['Documentdatum'].apply(set).reset_index(name='docdatums')
df_documentdatums = df_documentdatums[df_documentdatums['docdatums'].map(lambda x: len(x)) == 1]
df_documentdatums['docdatums'] = df_documentdatums['docdatums'].apply(lambda x: list(x)[0])

df_verzoekdatums = Dataframe.groupby(['Verzoek'])['verzoekdatum'].apply(list).reset_index(name='verdatums')
df_verantwoordelijke = Dataframe.groupby(['Verzoek'])['Verantwoordelijke'].apply(set).reset_index(name='verantwoordelijke')
df_verantwoordelijke['verantwoordelijke'] = df_verantwoordelijke['verantwoordelijke'].apply(lambda x: list(x)[0])

display(df_urls)
display(df_documentdatums)
display(df_verzoekdatums)

,Verzoek,urls
0,Beslissing op bezwaar Wob verzoek corresponden...,[https://open.overheid.nl/documenten/ronl-34e6...
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,[https://open.overheid.nl/documenten/ronl-7a0f...
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,[https://open.overheid.nl/documenten/ronl-0ba9...
3,Beslissing op bezwaar Wob-besluit Financiën ov...,[https://open.overheid.nl/documenten/ronl-e3e1...
4,Beslissing op bezwaar Wob-besluit over afstemm...,[https://open.overheid.nl/documenten/ronl-07e2...
...,...,...
1128,Besluit van Wob-verzoek over productverbetering,[https://open.overheid.nl/documenten/ronl-0491...
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,[https://open.overheid.nl/documenten/ronl-370d...
1130,Woo-verzoek over de identiteit achter anonieme...,[https://open.overheid.nl/documenten/ronl-2a0b...
1131,Woo-verzoek over landbouwvrijstelling in inkom...,[https://open.overheid.nl/documenten/ronl-1c09...


,Verzoek,docdatums
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09
...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27


,Verzoek,verdatums
0,Beslissing op bezwaar Wob verzoek corresponden...,"[nan, nan, nan]"
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,"[14-07-2022, nan, nan]"
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,"[image, 02-11-2021, 02-11-2021]"
3,Beslissing op bezwaar Wob-besluit Financiën ov...,"[22-10-2021, nan]"
4,Beslissing op bezwaar Wob-besluit over afstemm...,"[nan, 05-10-2022]"
...,...,...
1128,Besluit van Wob-verzoek over productverbetering,"[13-01-2022, 13-01-2022]"
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,"[22-10-2021, nan]"
1130,Woo-verzoek over de identiteit achter anonieme...,[02-08-2022]
1131,Woo-verzoek over landbouwvrijstelling in inkom...,"[nan, 07-08-2022, nan]"


In [109]:
display(df_verantwoordelijke)

,Verzoek,verantwoordelijke
0,Beslissing op bezwaar Wob verzoek corresponden...,Ministerie van Infrastructuur en Waterstaat
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,Ministerie van Infrastructuur en Waterstaat
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,"Ministerie van Landbouw, Natuur en Voedselkwal..."
3,Beslissing op bezwaar Wob-besluit Financiën ov...,Ministerie van Financiën
4,Beslissing op bezwaar Wob-besluit over afstemm...,Ministerie van Financiën
...,...,...
1128,Besluit van Wob-verzoek over productverbetering,"Ministerie van Volksgezondheid, Welzijn en Sport"
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,Ministerie van Justitie en Veiligheid
1130,Woo-verzoek over de identiteit achter anonieme...,"Ministerie van Landbouw, Natuur en Voedselkwal..."
1131,Woo-verzoek over landbouwvrijstelling in inkom...,Ministerie van Financiën


In [110]:
df_documentdatums = df_documentdatums.join(df_urls.set_index('Verzoek'), on='Verzoek')
df = df_documentdatums.join(df_verzoekdatums.set_index('Verzoek'), on='Verzoek')
dfII = df.join(df_verantwoordelijke.set_index('Verzoek'), on='Verzoek')

In [111]:
def datum_naar_kwartaal(datum):
    """" Functie om datum om te zetten naar kwartaal"""

    maand = int(datum[5:7])
    kwartaal = math.ceil(maand / 3)
    return f"{kwartaal}e Kwartaal"

In [112]:
dfII['jaar'] = dfII['docdatums'].astype(str).str.slice(0 , 4)
dfII['kwartaal'] = dfII['docdatums'].astype(str).map(datum_naar_kwartaal)
display(dfII)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,[https://open.overheid.nl/documenten/ronl-34e6...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,3e Kwartaal
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,"[14-07-2022, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,"[image, 02-11-2021, 02-11-2021]","Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,"[22-10-2021, nan]",Ministerie van Financiën,2022,2e Kwartaal
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,"[nan, 05-10-2022]",Ministerie van Financiën,2023,1e Kwartaal
...,...,...,...,...,...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,[https://open.overheid.nl/documenten/ronl-7517...,[image],Ministerie van Justitie en Veiligheid,2022,3e Kwartaal
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,"[22-10-2021, nan]",Ministerie van Justitie en Veiligheid,2022,2e Kwartaal
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,[02-08-2022],"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,"[nan, 07-08-2022, nan]",Ministerie van Financiën,2023,1e Kwartaal


In [113]:
display(dfII)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,[https://open.overheid.nl/documenten/ronl-34e6...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,3e Kwartaal
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,"[14-07-2022, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,"[image, 02-11-2021, 02-11-2021]","Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,"[22-10-2021, nan]",Ministerie van Financiën,2022,2e Kwartaal
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,"[nan, 05-10-2022]",Ministerie van Financiën,2023,1e Kwartaal
...,...,...,...,...,...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,[https://open.overheid.nl/documenten/ronl-7517...,[image],Ministerie van Justitie en Veiligheid,2022,3e Kwartaal
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,"[22-10-2021, nan]",Ministerie van Justitie en Veiligheid,2022,2e Kwartaal
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,[02-08-2022],"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,"[nan, 07-08-2022, nan]",Ministerie van Financiën,2023,1e Kwartaal


In [114]:
ls = []
for i in dfII.verdatums:
    if 'image' in i:
        ls.append(True)
    else:
        ls.append(False)
        
dfII['image'] = ls

display(dfII)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal,image
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,[https://open.overheid.nl/documenten/ronl-34e6...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,3e Kwartaal,False
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,"[14-07-2022, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal,False
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,"[image, 02-11-2021, 02-11-2021]","Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal,True
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,"[22-10-2021, nan]",Ministerie van Financiën,2022,2e Kwartaal,False
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,"[nan, 05-10-2022]",Ministerie van Financiën,2023,1e Kwartaal,False
...,...,...,...,...,...,...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,[https://open.overheid.nl/documenten/ronl-7517...,[image],Ministerie van Justitie en Veiligheid,2022,3e Kwartaal,True
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,"[22-10-2021, nan]",Ministerie van Justitie en Veiligheid,2022,2e Kwartaal,False
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,[02-08-2022],"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal,False
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,"[nan, 07-08-2022, nan]",Ministerie van Financiën,2023,1e Kwartaal,False


In [115]:
df = dfII.copy()

verdatums = []
verdatum = []
for verzoek in df['verdatums']:
    if type(verzoek) is not list:
        verdatums.append(verzoek)
    
    else:    
        datum = [i for i in verzoek if '-' in str(i)]
        if any(datum) == True:
            verdatums.append(verzoek)
            verdatum.append(datum[0])
        else:
            verdatums.append(verzoek)
            verdatum.append(np.nan)

df['verdatums'] = verdatums
df['verdatum'] = verdatum

display(df)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal,image,verdatum
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,[https://open.overheid.nl/documenten/ronl-34e6...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,3e Kwartaal,False,NaN
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,"[14-07-2022, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal,False,14-07-2022
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,"[image, 02-11-2021, 02-11-2021]","Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal,True,02-11-2021
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,"[22-10-2021, nan]",Ministerie van Financiën,2022,2e Kwartaal,False,22-10-2021
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,"[nan, 05-10-2022]",Ministerie van Financiën,2023,1e Kwartaal,False,05-10-2022
...,...,...,...,...,...,...,...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,[https://open.overheid.nl/documenten/ronl-7517...,[image],Ministerie van Justitie en Veiligheid,2022,3e Kwartaal,True,NaN
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,"[22-10-2021, nan]",Ministerie van Justitie en Veiligheid,2022,2e Kwartaal,False,22-10-2021
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,[02-08-2022],"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal,False,02-08-2022
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,"[nan, 07-08-2022, nan]",Ministerie van Financiën,2023,1e Kwartaal,False,07-08-2022


In [116]:
XXX = df.copy()
XXX = XXX.drop_duplicates(subset = ['Verzoek'])
XXX = XXX.groupby(['verantwoordelijke'])['verantwoordelijke'].count().reset_index(name='aantal')

display(XXX)

# Data1.to_csv(r'C:\Users\ramon\Downloads\TestPlot.csv')

,verantwoordelijke,aantal
0,Ministerie van Algemene Zaken,3
1,Ministerie van Binnenlandse Zaken en Koninkrij...,96
2,Ministerie van Buitenlandse Zaken,69
3,Ministerie van Defensie,48
4,Ministerie van Economische Zaken en Klimaat,88
5,Ministerie van Financiën,150
6,Ministerie van Infrastructuur en Waterstaat,117
7,Ministerie van Justitie en Veiligheid,129
8,"Ministerie van Landbouw, Natuur en Voedselkwal...",281
9,"Ministerie van Onderwijs, Cultuur en Wetenschap",52


In [117]:
display(df)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal,image,verdatum
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,[https://open.overheid.nl/documenten/ronl-34e6...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,3e Kwartaal,False,NaN
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,"[14-07-2022, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal,False,14-07-2022
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,"[image, 02-11-2021, 02-11-2021]","Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal,True,02-11-2021
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,"[22-10-2021, nan]",Ministerie van Financiën,2022,2e Kwartaal,False,22-10-2021
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,"[nan, 05-10-2022]",Ministerie van Financiën,2023,1e Kwartaal,False,05-10-2022
...,...,...,...,...,...,...,...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,[https://open.overheid.nl/documenten/ronl-7517...,[image],Ministerie van Justitie en Veiligheid,2022,3e Kwartaal,True,NaN
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,"[22-10-2021, nan]",Ministerie van Justitie en Veiligheid,2022,2e Kwartaal,False,22-10-2021
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,[02-08-2022],"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal,False,02-08-2022
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,"[nan, 07-08-2022, nan]",Ministerie van Financiën,2023,1e Kwartaal,False,07-08-2022


In [118]:
df_dates = df[df['verdatum'].apply(lambda x: isinstance(x, str))]
display(df_dates)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal,image,verdatum
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,"[14-07-2022, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal,False,14-07-2022
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,"[image, 02-11-2021, 02-11-2021]","Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal,True,02-11-2021
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,"[22-10-2021, nan]",Ministerie van Financiën,2022,2e Kwartaal,False,22-10-2021
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,"[nan, 05-10-2022]",Ministerie van Financiën,2023,1e Kwartaal,False,05-10-2022
5,Beslissing op bezwaar Wob-besluit over contact...,2023-02-08,[https://open.overheid.nl/documenten/ronl-aa6e...,"[25-03-2021, nan, image]",Ministerie van Financiën,2023,1e Kwartaal,True,25-03-2021
...,...,...,...,...,...,...,...,...,...
1125,Besluit over koude weer training,2022-09-13,[https://open.overheid.nl/documenten/ronl-60da...,[09-06-2022],Ministerie van Defensie,2022,3e Kwartaal,False,09-06-2022
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,"[22-10-2021, nan]",Ministerie van Justitie en Veiligheid,2022,2e Kwartaal,False,22-10-2021
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,[02-08-2022],"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal,False,02-08-2022
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,"[nan, 07-08-2022, nan]",Ministerie van Financiën,2023,1e Kwartaal,False,07-08-2022


In [119]:
# url = 'https://open.overheid.nl/documenten/ronl-7a0f146a99fbedda2e677881e1516d02894733c9/pdf'
# res = requests.get(url)
# html_page = res.content

# soup = BeautifulSoup(html_page, 'html.parser')

# text = soup.find_all()
# print(text)

In [120]:
df_dates['docdatums'] = pd.to_datetime(df_dates['docdatums'], format = '%d-%m-%Y')
df_dates['verdatums'] = pd.to_datetime(df_dates['verdatum'], format = '%d-%m-%Y')


df_dates['dagen'] = (df_dates['docdatums'] - df_dates['verdatums']).dt.days
df_dates = df_dates[df_dates['dagen'] > 0]

display(df_dates)

#df_dates.to_csv(r'C:\Users\ramon\Downloads\PlotData.csv', index=False)

C:\Users\ramon\AppData\Local\Temp\ipykernel_33692\4176417851.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ramon\AppData\Local\Temp\ipykernel_33692\4176417851.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ramon\AppData\Local\Temp\ipykernel_33692\4176417851.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal,image,verdatum,dagen
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,2022-07-14,Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal,False,14-07-2022,162
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,2021-11-02,"Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal,True,02-11-2021,451
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,2021-10-22,Ministerie van Financiën,2022,2e Kwartaal,False,22-10-2021,186
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,2022-10-05,Ministerie van Financiën,2023,1e Kwartaal,False,05-10-2022,127
5,Beslissing op bezwaar Wob-besluit over contact...,2023-02-08,[https://open.overheid.nl/documenten/ronl-aa6e...,2021-03-25,Ministerie van Financiën,2023,1e Kwartaal,True,25-03-2021,685
...,...,...,...,...,...,...,...,...,...,...
1125,Besluit over koude weer training,2022-09-13,[https://open.overheid.nl/documenten/ronl-60da...,2022-06-09,Ministerie van Defensie,2022,3e Kwartaal,False,09-06-2022,96
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,2021-10-22,Ministerie van Justitie en Veiligheid,2022,2e Kwartaal,False,22-10-2021,251
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,2022-08-02,"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal,False,02-08-2022,23
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,2022-08-07,Ministerie van Financiën,2023,1e Kwartaal,False,07-08-2022,204


De documentdatum van deze twee is gewoon volledig fout. En is de datum van het verzoek de documentdatum geworden terwijl het besluitdocument pas weken/maanden later uit is gekomen.

In [121]:
Data1 = df_dates.copy()
Data1 = Data1.drop_duplicates(subset = ['Verzoek'])
Data1 = Data1.groupby(['verantwoordelijke'])['verantwoordelijke'].count().reset_index(name='aantal')

display(Data1)

XXX=XXX.join(Data1.set_index('verantwoordelijke'), on='verantwoordelijke', how='left', lsuffix='_left', rsuffix='_right')
Merged = XXX.fillna(0)

display(Merged)

,verantwoordelijke,aantal
0,Ministerie van Algemene Zaken,3
1,Ministerie van Binnenlandse Zaken en Koninkrij...,87
2,Ministerie van Buitenlandse Zaken,64
3,Ministerie van Defensie,36
4,Ministerie van Economische Zaken en Klimaat,76
5,Ministerie van Financiën,129
6,Ministerie van Infrastructuur en Waterstaat,89
7,Ministerie van Justitie en Veiligheid,62
8,"Ministerie van Landbouw, Natuur en Voedselkwal...",247
9,"Ministerie van Onderwijs, Cultuur en Wetenschap",39


,verantwoordelijke,aantal_left,aantal_right
0,Ministerie van Algemene Zaken,3,3
1,Ministerie van Binnenlandse Zaken en Koninkrij...,96,87
2,Ministerie van Buitenlandse Zaken,69,64
3,Ministerie van Defensie,48,36
4,Ministerie van Economische Zaken en Klimaat,88,76
5,Ministerie van Financiën,150,129
6,Ministerie van Infrastructuur en Waterstaat,117,89
7,Ministerie van Justitie en Veiligheid,129,62
8,"Ministerie van Landbouw, Natuur en Voedselkwal...",281,247
9,"Ministerie van Onderwijs, Cultuur en Wetenschap",52,39


In [122]:
Merged['difference'] = Merged['aantal_left'] - Merged['aantal_right']
Merged['percentage'] = Merged['aantal_right'] / Merged['aantal_left']
display(Merged)

print((sum(Merged.aantal_right) + 50) /sum(Merged.aantal_left))

,verantwoordelijke,aantal_left,aantal_right,difference,percentage
0,Ministerie van Algemene Zaken,3,3,0,1.000000
1,Ministerie van Binnenlandse Zaken en Koninkrij...,96,87,9,0.906250
2,Ministerie van Buitenlandse Zaken,69,64,5,0.927536
3,Ministerie van Defensie,48,36,12,0.750000
4,Ministerie van Economische Zaken en Klimaat,88,76,12,0.863636
5,Ministerie van Financiën,150,129,21,0.860000
6,Ministerie van Infrastructuur en Waterstaat,117,89,28,0.760684
7,Ministerie van Justitie en Veiligheid,129,62,67,0.480620
8,"Ministerie van Landbouw, Natuur en Voedselkwal...",281,247,34,0.879004
9,"Ministerie van Onderwijs, Cultuur en Wetenschap",52,39,13,0.750000


0.8282374100719424


In [123]:
# simpel plotje die al met de huidige data te maken is

fig = px.bar(y=Data1.verantwoordelijke, x=Data1.aantal, text_auto='1s',
            title="Aantal WOO/WOB verzoeken per Ministerie")
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [27]:
df2 = df_dates.copy()
df2 = df2.groupby(['verantwoordelijke'])['dagen'].mean().reset_index(name='dagen')

display(df2)

fig = px.bar(x=df2.verantwoordelijke, y=df2.dagen)
fig.show()

,verantwoordelijke,dagen
0,Ministerie van Algemene Zaken,89.666667
1,Ministerie van Binnenlandse Zaken en Koninkrij...,144.156627
2,Ministerie van Buitenlandse Zaken,148.746032
3,Ministerie van Defensie,91.944444
4,Ministerie van Economische Zaken en Klimaat,190.853333
5,Ministerie van Financiën,248.875000
6,Ministerie van Infrastructuur en Waterstaat,250.488372
7,Ministerie van Justitie en Veiligheid,212.833333
8,"Ministerie van Landbouw, Natuur en Voedselkwal...",192.537500
9,"Ministerie van Onderwijs, Cultuur en Wetenschap",129.317073


In [28]:
imglist = []

for i in dfII.verdatums:
    for ele in i:
        imglist.append(ele)
        
images = Counter(imglist)['image']
length = len(imglist)

print((images/length)*100)

12.11667913238594


In [29]:
# for i, r in dfII.iterrows():
#     if r.verantwoordelijke == 'Ministerie van Justitie en Veiligheid':
#         print(r.Verzoek)
#         print(len(r.urls))
#         print(r.verdatums)

In [30]:
# LIST = []
# for i, r in dfII.iterrows():
#     lsst = []
#     if r.verantwoordelijke == 'Ministerie van Justitie en Veiligheid':
#         for url in r.urls:
#             print(url)
#             lsst.append(verzoek_datum(url))
#         LIST.append(lsst)
        
# print(LIST)

In [32]:
tem = []
print(LIST)

for I in LIST:
    tem2 = []
    for i in I:
        if '-' in str(i):
            tem2.append(True)
        else:
            tem2.append(False)
        
    if any(tem2):
        tem.append(True)
    else:
        tem.append(False)

print(sum(tem)/len(tem))
            
        

NameError: name 'LIST' is not defined

83.33

https://open.overheid.nl/documenten/ronl-ff8822d30a30c1f70df73b7394eb941503f04728/pdf In dit document is het jaar weggelaten uit van het verzoek.

In [33]:
dfIII = dfII.copy()
dfIII = dfIII.groupby(['verantwoordelijke'])['image'].apply(list).reset_index(name='image')

means = [i.count(True) / len(i) for i in dfIII.image]

dfIII['image'] = means
display(dfIII)

fig = px.bar(x=dfIII.verantwoordelijke, y=dfIII.image, hover_data = [Data1['aantal']])
fig.show()

,verantwoordelijke,image
0,Ministerie van Algemene Zaken,0.333333
1,Ministerie van Binnenlandse Zaken en Koninkrij...,0.010417
2,Ministerie van Buitenlandse Zaken,0.086957
3,Ministerie van Defensie,0.062500
4,Ministerie van Economische Zaken en Klimaat,0.386364
5,Ministerie van Financiën,0.066667
6,Ministerie van Infrastructuur en Waterstaat,0.196581
7,Ministerie van Justitie en Veiligheid,0.403101
8,"Ministerie van Landbouw, Natuur en Voedselkwal...",0.135231
9,"Ministerie van Onderwijs, Cultuur en Wetenschap",0.076923


In [34]:
result = dfII.copy()
result['periode'] = result['jaar'] + ' ' + result['kwartaal']
temp_result = result.groupby(['periode','verantwoordelijke']).count()
temp_result['image'] = result.groupby(['periode','verantwoordelijke'])['image'].sum()
result = temp_result

result = result.drop(['docdatums', 'urls', 'verdatums', 'jaar', 'kwartaal'], axis=1)
result = result.reset_index(level=['periode', 'verantwoordelijke'])
result['temp'] = result.periode + ' ' + result.verantwoordelijke
periodes = list(set(i for i in result.periode))
vera = list(set(i for i in result.verantwoordelijke))
temp_list = result.temp.values

new_rows = []

for i in periodes:
    for n in vera:
        value = i+' '+n
        
        if value not in temp_list:
            new_rows.append([i, n, 0, 0, np.nan])
            
Length = len(result)
for i in enumerate(new_rows):
    result.loc[Length + i[0]] = i[1]
    
result = result.drop(['temp'], axis=1)

result = result.sort_values(['periode', 'verantwoordelijke'],
              ascending = [True, True]).reset_index()

result = result.drop(['index'], axis=1)
        
display(result)        

,periode,verantwoordelijke,Verzoek,image
0,2022 1e Kwartaal,Ministerie van Algemene Zaken,0,0
1,2022 1e Kwartaal,Ministerie van Binnenlandse Zaken en Koninkrij...,0,0
2,2022 1e Kwartaal,Ministerie van Buitenlandse Zaken,1,0
3,2022 1e Kwartaal,Ministerie van Defensie,0,0
4,2022 1e Kwartaal,Ministerie van Economische Zaken en Klimaat,2,1
5,2022 1e Kwartaal,Ministerie van Financiën,1,0
6,2022 1e Kwartaal,Ministerie van Infrastructuur en Waterstaat,8,1
7,2022 1e Kwartaal,Ministerie van Justitie en Veiligheid,0,0
8,2022 1e Kwartaal,"Ministerie van Landbouw, Natuur en Voedselkwal...",3,1
9,2022 1e Kwartaal,"Ministerie van Onderwijs, Cultuur en Wetenschap",1,0


In [35]:
fig = px.bar(x=dfIII.verantwoordelijke, y=dfIII.image)
fig.show()

In [36]:
result['pdf'] = result['Verzoek'] - result['image']
display(result)

,periode,verantwoordelijke,Verzoek,image,pdf
0,2022 1e Kwartaal,Ministerie van Algemene Zaken,0,0,0
1,2022 1e Kwartaal,Ministerie van Binnenlandse Zaken en Koninkrij...,0,0,0
2,2022 1e Kwartaal,Ministerie van Buitenlandse Zaken,1,0,1
3,2022 1e Kwartaal,Ministerie van Defensie,0,0,0
4,2022 1e Kwartaal,Ministerie van Economische Zaken en Klimaat,2,1,1
5,2022 1e Kwartaal,Ministerie van Financiën,1,0,1
6,2022 1e Kwartaal,Ministerie van Infrastructuur en Waterstaat,8,1,7
7,2022 1e Kwartaal,Ministerie van Justitie en Veiligheid,0,0,0
8,2022 1e Kwartaal,"Ministerie van Landbouw, Natuur en Voedselkwal...",3,1,2
9,2022 1e Kwartaal,"Ministerie van Onderwijs, Cultuur en Wetenschap",1,0,1


In [37]:
fig = px.bar(result, x="verantwoordelijke", y=["image", "pdf"], animation_frame='periode',
         hover_name="verantwoordelijke")

fig.update_yaxes(
    autorange = True,
    fixedrange = True
)


fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()

In [38]:
result2 = result.groupby(['verantwoordelijke'])['pdf','image'].sum().reset_index()
display(result2)

C:\Users\ramon\AppData\Local\Temp\ipykernel_33692\1376667530.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,verantwoordelijke,pdf,image
0,Ministerie van Algemene Zaken,2,1
1,Ministerie van Binnenlandse Zaken en Koninkrij...,95,1
2,Ministerie van Buitenlandse Zaken,63,6
3,Ministerie van Defensie,45,3
4,Ministerie van Economische Zaken en Klimaat,54,34
5,Ministerie van Financiën,140,10
6,Ministerie van Infrastructuur en Waterstaat,94,23
7,Ministerie van Justitie en Veiligheid,77,52
8,"Ministerie van Landbouw, Natuur en Voedselkwal...",243,38
9,"Ministerie van Onderwijs, Cultuur en Wetenschap",48,4


In [39]:
import plotly.graph_objects as go

ver = result.verantwoordelijke.values

fig = go.Figure()
fig.add_trace(go.Bar(
    x=ver,
    y=result2.pdf,
    name='text-pdf',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=ver,
    y=result2.image,
    name='fotos',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [135]:
count = 0
count2 = 0

for i in LIST:
    count2 +=1
    X = [ele for ele in i if '-' in str(ele)]
    if X != []:
        count+=1
    else:
        print(count2)
        
print(count/len(LIST))

2
5
8
9
24
28
0.875


In [160]:
display(df)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal,image,verdatum
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,[https://open.overheid.nl/documenten/ronl-34e6...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,3e Kwartaal,False,NaN
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,"[14-07-2022, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal,False,14-07-2022
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,"[image, 02-11-2021, 02-11-2021]","Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal,True,02-11-2021
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,"[22-10-2021, nan]",Ministerie van Financiën,2022,2e Kwartaal,False,22-10-2021
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,"[nan, 05-10-2022]",Ministerie van Financiën,2023,1e Kwartaal,False,05-10-2022
...,...,...,...,...,...,...,...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,[https://open.overheid.nl/documenten/ronl-7517...,[image],Ministerie van Justitie en Veiligheid,2022,3e Kwartaal,True,NaN
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,"[22-10-2021, nan]",Ministerie van Justitie en Veiligheid,2022,2e Kwartaal,False,22-10-2021
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,[02-08-2022],"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal,False,02-08-2022
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,"[nan, 07-08-2022, nan]",Ministerie van Financiën,2023,1e Kwartaal,False,07-08-2022


In [163]:
verzoek_datums = []

count = 0
for i,r in df.iterrows():
    if r.image == True and r.verdatum is np.nan:
        count+=1
        print(count)
        vds = []
        for idx, value in enumerate(r.verdatums):
            if value == 'image':
                vd = verzoek_datum(r.urls[idx], True)
                print(vd)
                print(r.urls[idx])
                vds.append(vd)          
                
    else:
        vds = np.nan
    
    if vds != np.nan:
        print(vds)
        
    verzoek_datums.append(vds)

nan
nan
nan
nan
nan
nan
nan
nan
nan
1
(nan, 73)
https://open.overheid.nl/documenten/ronl-62c1905bc371b853e52acd3a7f0d3dba6b090d20/pdf
[(nan, 73)]
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2
('24-02-2022', 5)
https://open.overheid.nl/documenten/ronl-d82e831e1472117eafa560f51ff30202ea2fda3b/pdf
[('24-02-2022', 5)]
3
('09-05-2022', 6)
https://open.overheid.nl/documenten/ronl-41bda038b24f871db89186d5dedbcc891fcd24fe/pdf
[('09-05-2022', 6)]
nan
nan
nan
nan
nan
nan
4
(nan, 28)
https://open.overheid.nl/documenten/ronl-6dce40798a138a5a0a77a868448822a3307a6557/pdf
[(nan, 28)]
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
5
(nan, 56)
https://open.overheid.nl/documenten/ronl-034d6865b2267014396b2a70ecc499a9bb60adf3/pdf
('10-05-2021', 5)
https://open.overheid.nl/documenten/ronl-d3a04859846ecb0609d85e869459139ebdd6e1d9/pdf
[(nan, 56), ('

('06-09-2022', 7)
https://open.overheid.nl/documenten/ronl-f47eec8a7a10ecf56c3fd7ae146b5041f3bacae7/pdf
[(nan, 95), (nan, 438), (nan, 6), (nan, 4), ('06-09-2022', 7)]
nan
nan
nan
nan
nan
nan
nan
nan
nan
36
(nan, 2)
https://open.overheid.nl/documenten/ronl-4883a7e8c9275999c59f136ed931ca31be18dc41/pdf
(nan, 98)
https://open.overheid.nl/documenten/ronl-b4b19d95907895c6078efcbb360abe64e42af95c/pdf
[(nan, 2), (nan, 98)]
nan
37
('10-06-2022', 4)
https://open.overheid.nl/documenten/ronl-d88571189cbd9a865416729bee4e58adb8b06441/pdf
[('10-06-2022', 4)]
nan
nan
nan
38
(nan, 16)
https://open.overheid.nl/documenten/ronl-059cafe348a226a30708c812268e51b104e137e4/pdf
[(nan, 16)]
nan
nan
39
('12-12-2022', 3)
https://open.overheid.nl/documenten/ronl-c7c5fd16a1652e18ab92f6cea3e80a074990dfd0/pdf
[('12-12-2022', 3)]
40
('12-12-2022', 3)
https://open.overheid.nl/documenten/ronl-2b23e5d8309d176d5d68640929dd0ccd69cf63da/pdf
[('12-12-2022', 3)]
nan
41
('08-06-2022', 3)
https://open.overheid.nl/documenten/ronl

In [164]:
print(verzoek_datums)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, [(nan, 73)], nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, [('24-02-2022', 5)], [('09-05-2022', 6)], nan, nan, nan, nan, nan, nan, [(nan, 28)], nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, [(nan, 56), ('10-05-2021', 5)], nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, [(nan, 4)], nan, nan, [('23-11-2021', 2), (nan, 1)], nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, [('02-12-2021', 59)], nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, [(nan, 5), ('11-11-2021', 3)], nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, [(nan, 53)], nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

In [175]:
df_withOCR = pd.read_csv('WithOCR.csv')
display(df_withOCR)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal,image,verdatum,OCR_datum
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,['https://open.overheid.nl/documenten/ronl-34e...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,3e Kwartaal,False,NaN,NaN
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,['https://open.overheid.nl/documenten/ronl-7a0...,"['14-07-2022', nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal,False,14-07-2022,NaN
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,['https://open.overheid.nl/documenten/ronl-0ba...,"['image', '02-11-2021', '02-11-2021']","Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal,True,02-11-2021,NaN
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,['https://open.overheid.nl/documenten/ronl-e3e...,"['22-10-2021', nan]",Ministerie van Financiën,2022,2e Kwartaal,False,22-10-2021,NaN
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,['https://open.overheid.nl/documenten/ronl-07e...,"[nan, '05-10-2022']",Ministerie van Financiën,2023,1e Kwartaal,False,05-10-2022,NaN
...,...,...,...,...,...,...,...,...,...,...
1107,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,['https://open.overheid.nl/documenten/ronl-751...,['image'],Ministerie van Justitie en Veiligheid,2022,3e Kwartaal,True,NaN,22-03-2022
1108,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,['https://open.overheid.nl/documenten/ronl-370...,"['22-10-2021', nan]",Ministerie van Justitie en Veiligheid,2022,2e Kwartaal,False,22-10-2021,NaN
1109,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,['https://open.overheid.nl/documenten/ronl-2a0...,['02-08-2022'],"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal,False,02-08-2022,NaN
1110,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,['https://open.overheid.nl/documenten/ronl-1c0...,"[nan, '07-08-2022', nan]",Ministerie van Financiën,2023,1e Kwartaal,False,07-08-2022,NaN


In [172]:
OCR_datum = []
count1 = 0
count2 = 0
for i in verzoek_datums:
    if isinstance(i, list):
        datum = [ele for ele in i if '-' in str(ele)]
        count1 += 1
        if len(datum) > 0:
            count2 += 1
            OCR_datum.append(datum[0][0])
        else:
            OCR_datum.append(np.nan)
    else:
        OCR_datum.append(np.nan)
        
        
print(count1)
print(count2)

print(count2/count1)
        
df['OCR_datum'] = OCR_datum

70
51
0.7285714285714285


In [173]:
#df.to_csv(r'C:\Users\ramon\Downloads\WithOCR.csv', index = False)

In [167]:
for i, r in df.iterrows():
    if r.OCR_datum is np.nan and r.verdatum is np.nan:
        for i in r.urls:
            print(i)

https://open.overheid.nl/documenten/ronl-34e6543c6a831746aeca995b23f6e546938e0f43/pdf
https://open.overheid.nl/documenten/ronl-40e537aa5e5214fa7bf8aeb65ad23773e42867c7/pdf
https://open.overheid.nl/documenten/ronl-4f5cc3ca1ec6072b43d21bf02472e7a28c206958/pdf
https://open.overheid.nl/documenten/ronl-277c9d700e93a01e9941268fbf740d6bcf7ca74b/pdf
https://open.overheid.nl/documenten/ronl-3d20e775b530e8e71dea23d1f36920f85e55e2a0/pdf
https://open.overheid.nl/documenten/ronl-62c1905bc371b853e52acd3a7f0d3dba6b090d20/pdf
https://open.overheid.nl/documenten/ronl-fa05a8e4a3eebf1ee165789fffb8bbd3738f9c0b/pdf
https://open.overheid.nl/documenten/ronl-f87f559f51dfd236ed49371f031e1a52a87ab163/pdf
https://open.overheid.nl/documenten/ronl-23385913eccddf504486cc832f76850e67128408/pdf
https://open.overheid.nl/documenten/ronl-6dce40798a138a5a0a77a868448822a3307a6557/pdf
https://open.overheid.nl/documenten/ronl-0a515d1bb923daf03ce8c9397bce5df5c4b4f9b2/pdf
https://open.overheid.nl/documenten/ronl-7514a8fdb9a4d

In [183]:
Verantw = set(df_withOCR.verantwoordelijke.values.tolist())
print(Verantw)   

{'Ministerie van Financiën', 'Ministerie van Onderwijs, Cultuur en Wetenschap', 'Ministerie van Algemene Zaken', 'Ministerie van Economische Zaken en Klimaat', 'Ministerie van Binnenlandse Zaken en Koninkrijksrelaties', 'Ministerie van Defensie', 'Ministerie van Landbouw, Natuur en Voedselkwaliteit', 'Ministerie van Buitenlandse Zaken', 'Ministerie van Infrastructuur en Waterstaat', 'Ministerie van Sociale Zaken en Werkgelegenheid', 'Ministerie van Volksgezondheid, Welzijn en Sport', 'Ministerie van Justitie en Veiligheid'}


In [190]:
counter_nn = 0

for vera in Verantw:
    for i,r in df_withOCR.iterrows():
        if r.verantwoordelijke == vera:
            dates = [r.OCR_datum, r.verdatum]
            non_nan_values = [ele for ele in dates if ele is not np.nan]

            if non_nan_values:
                counter_nn += 1
            
print(counter_nn)
print(len(df_withOCR))

print(counter_nn/len(df_withOCR))

969
1112
0.8714028776978417
